# Glassco

Glassco manufactures wine glasses, beer glasses, champagne glasses and whiskey glasses.  Each type of glass uses time in the molding shop, time in the packaging shop, and a certain amount of glass.  The resources required to make each type of glass are given in the following table:

|Data          |WINE GLASS|BEER GLASS|CHMPGNE GLASS|WHISKEY GLASS|
|--------------|----------|----------|-------------|-------------|
|Molding time  |4 minutes |9 minutes |7 minutes    |10 minutes   |
|Packaging time|1 minute  |1 minute  |3 minutes    |40 minutes   |
|Glass         |3 oz      |4 oz      |2 oz         |1 oz         |
|Selling price |\$6       |\$10      |\$9          |\$20         |

At present, 600 minutes of molding time, 400 minutes of packaging time and 500 oz of glass are available.  

### Write down the LP (in GAMS) that Glassco should solve, assuming the company wishes to maximize revenue.

In [1]:
# Load the gams extension
%load_ext gams_magic

In [2]:
%%gams
set j /WINE, BEER, CHMPGNE, WHISKEY/;
set i /Molding, Packaging, Glass/;

table Data(*,j)
               WINE BEER CHMPGNE WHISKEY 
Molding          4    9     7      10
Packaging        1    1     3      40
Glass            3    4     2       1
price            6   10     9      20;

parameter b(i)
    /Molding 600
     Packaging 400
     Glass 500/;
     
parameter c(j);
c(j) = Data("price",j);

parameter A(i,j);
A(i,j) = Data(i,j);

nonnegative variables x(j);
variable z;

equations rprimal(i), obj;
rprimal(i).. sum(j, A(i,j)*x(j) ) =L= b(i) ;
obj.. z =E= sum(j, c(j)*x(j) );

model primal /rprimal, obj/;
solve primal using LP maximizing z;
display z.l;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),933.3333,4,5,LP,CPLEX,0.047


### Write down the dual of this LP problem, in the same GAMS file.
You should set up two separate models and include just those equations needed in each model in the model statement.

In [3]:
%%gams
nonnegative variables x(j), y(i);
variable zprimal;
variable zdual;

equations rprimal(i), rdual(j), objprimal, objdual;
rprimal(i).. sum(j, A(i,j)*x(j) ) =L= b(i) ;
rdual(j).. sum(i, A(i,j)*y(i) ) =G= c(j) ;

objprimal.. zprimal =E= sum(j, c(j)*x(j) );
objdual.. zdual =E= sum(i, b(i)*y(i) );

*model primal /rprimal, objprimal/;
*solve primal using LP maximizing zprimal;
*display zprimal.l;

model dual /rdual, objdual/;
solve dual using LP minimizing zdual;
display zdual.l;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),933.3333,5,4,LP,CPLEX,0


### Glassco claims that the optimal solution to the (primal) problem is $z = 2800/3$, $x_1 = 400/3$, $x_2 = 0$, $x_3 = 0$ and $x_4 = 20/3$. Is this the case?  
Justify, and quote any relevant theory you use accurately.

In [4]:
# demonstrate here or in above text box
%gams_pull -d zdual rdual

z = 2800/3
x1 = 400/3
x2 = 0
x3 = 0
x4 = 20/3

#print(rdual)
print("zdual:", round(zdual.level.values.item(0),5), "and z:", round(z,5) )
print("Marginal of restriction 'WINE':", round(rdual.loc[rdual['j']=='WINE'].marginal.values.item(0),5), "and x1:", round(x1,5) )
print("Marginal of restriction 'BEER':", round(rdual.loc[rdual['j']=='BEER'].marginal.values.item(0),5), "and x2:", round(x2,5) )
print("Marginal of restriction 'CHMPGNE':", round(rdual.loc[rdual['j']=='CHMPGNE'].marginal.values.item(0),5), "and x3:", round(x3,5) )
print("Marginal of restriction 'WHISKEY':", round(rdual.loc[rdual['j']=='WHISKEY'].marginal.values.item(0),5), "and x4:", round(x4,5) )


zdual: 933.33333 and z: 933.33333
Marginal of restriction 'WINE': 133.33333 and x1: 133.33333
Marginal of restriction 'BEER': 0.0 and x2: 0
Marginal of restriction 'CHMPGNE': 0.0 and x3: 0
Marginal of restriction 'WHISKEY': 6.66667 and x4: 6.66667


### What is the solution of the dual problem?  Can you show how the multipliers on the primal problem are related to the dual solution?

In [5]:
# dual solution here
%gams_pull -d zdual y
#print(y)
y1 = y.loc[y['i']=='Molding'].level.values.item(0)
y2 = y.loc[y['i']=='Packaging'].level.values.item(0)
y3 = y.loc[y['i']=='Glass'].level.values.item(0)

print("zdual:", round(zdual.level.values.item(0),5) )
print("Dual solution for 'Molding':", y1 )
print("Dual solution for 'Packaging':", y2 )
print("Dual solution for 'Glass':", y3 )


zdual: 933.33333
Dual solution for 'Molding': 1.4666666666666666
Dual solution for 'Packaging': 0.13333333333333336
Dual solution for 'Glass': 0.0


In [6]:
# demonstrate how related here (maybe also use markdown cell)
%gams_pull -d rprimal
#print(rprimal)
print("Marginal of restriction 'Molding':", round(rprimal.loc[rprimal['i']=='Molding'].marginal.values.item(0),5), "and y1:", round(y1,5) )
print("Marginal of restriction 'Packaging':", round(rprimal.loc[rprimal['i']=='Packaging'].marginal.values.item(0),5), "and y2:", round(y2,5) )
print("Marginal of restriction 'Glass':", round(rprimal.loc[rprimal['i']=='Glass'].marginal.values.item(0),5), "and y3:", round(y3,5) )


Marginal of restriction 'Molding': 1.46667 and y1: 1.46667
Marginal of restriction 'Packaging': 0.13333 and y2: 0.13333
Marginal of restriction 'Glass': 0.0 and y3: 0.0
